# Preprocessing: BJH039


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH039'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 3639799 =   1043.000 ...  1819.899 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 3639799 =   1043.000 ...  1819.899 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AR1 AR2
AR2 AR3
AR3 AR4
AR4 AR5
REF1 noref
REF2 noref
AR5 AR6
AR6 AR7
AR7 AR8
AR8 AR9
AR9 AR10
AR10 AR11
AR11 AR12
BR1 BR2
BR2 BR3
BR3 BR4
BR4 BR5
BR5 BR6
BR6 BR7
BR7 BR8
BR8 BR9
BR9 BR10
BR10 BR11
BR11 BR12
BR12 BR13
BR13 BR14
CR1 CR2
CR2 CR3
CR3 CR4
CR4 CR5
CR5 CR6
CR6 CR7
CR7 CR8
CR8 CR9
CR9 CR10
DR1 DR2
DR2 DR3
DR3 DR4
DR4 DR5
DR5 DR6
DR6 DR7
DR7 DR8
DR8 DR9
DR9 DR10
DR10 DR11
DR11 DR12
ER1 ER2
ER2 ER3
ER3 ER4
ER4 ER5
ER5 ER6
ER6 ER7
ER7 ER8
ER8 ER9
ER9 ER10
ER10 ER11
ER11 ER12
ER12 ER13
ER13 ER14
FR1 FR2
FR2 FR3
FR3 FR4
FR4 FR5
FR5 FR6
FR6 FR7
FR7 FR8
FR8 FR9
FR9 FR10
GR1 noref
GR2 GR3
GR3 GR4
GR4 GR5
GR5 GR6
GR6 GR7
GR7 GR8
GR8 GR9
GR9 GR10
GR10 GR11
GR11 GR12
GR12 GR13
GR13 GR14
HR1 HR2
HR2 HR3
HR3 HR4
HR4 HR5
HR5 HR6
HR6 HR7
HR7 HR8
HR8 HR9
HR9 HR10
HR10 HR11
HR11 HR12
IR1 IR2
IR2 IR3
IR3 IR5
IR4 noref
IR5 IR6
IR6 IR7
IR7 IR8
IR8 IR9
IR9 IR10
IR10 IR11
IR11 IR12
IR12 IR13
IR13 IR14
JR1 JR2
JR2 JR3
JR3 JR4
JR4 JR5
JR5 JR6
JR6 JR7
JR7 JR8
KL1 noref
KL2 noref
KL3 noref
KL4 noref
K

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 3639799  =      0.000 ...  1819.899 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=125, n_times=3639800
    Range : 0 ... 3639799 =      0.000 ...  1819.899 secs
Ready.
Added the following bipolar channels:
AR1-AR2, AR2-AR3, AR3-AR4, AR4-AR5, AR5-AR6, AR6-AR7, AR7-AR8, AR8-AR9, AR9-AR10, AR10-AR11, AR11-AR12, BR1-BR2, BR2-BR3, BR3-BR4, BR4-BR5, BR5-BR6, BR6-BR7, BR7-BR8, BR8-BR9, BR9-BR10, BR10-BR11, BR11-BR12, BR12-BR13, BR13-BR14, CR1-CR2, CR2-CR3, CR3-CR4, CR4-CR5, CR5-CR6, CR6-CR7, CR7-CR8, CR8-CR9, CR9-CR10, DR1-DR2, DR2-DR3, DR3-DR4, DR4-DR5, DR5-DR6, DR6-DR7, DR7-DR8, DR8-DR9, DR9-DR10, DR10-DR11, DR11-DR12, ER1-ER2, ER2-ER3, ER3-ER4, ER4-ER5, ER5-ER6, ER6-ER7, ER7-ER8, ER8-ER9, ER9-ER10, ER10-ER11, ER11-ER12, ER12-ER13, ER13-ER14, FR1-FR2, FR2-FR3, FR3-FR4, FR4-FR5, FR5-FR6, FR6-FR7, FR7-FR8, FR8-FR9, FR9-FR10, GR2-GR3, GR3-GR4, GR4-GR5, GR5-GR6, GR6-GR7, GR7-GR8, GR8-GR9, GR9-GR10, GR10-GR11, GR11-GR12, GR12-

/tmp/ipykernel_9070/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif
[done]
Reading 0 ... 3639799  =      0.000 ...  1819.899 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=125, n_times=3639800
    Range : 0 ... 3639799 =      0.000 ...  1819.899 secs
Ready.
Added the following bipolar channels:
AR1-AR2, AR2-AR3, AR3-AR4, AR4-AR5, AR5-AR6, AR6-AR7, AR7-AR8, AR8-AR9, AR9-AR10, AR10-AR11, AR11-AR12, BR1-BR2, BR2-BR3, BR3-BR4, BR4-BR5, BR5-BR6, BR6-BR7, BR7-BR8, BR8-BR9, BR9-BR10, BR10-BR11, BR11-BR12, BR12-BR13, BR13-BR14, CR1-CR2, CR2-CR3, CR3-CR4, CR4-CR5, CR5-CR6, CR6-CR7, CR7-CR8, CR8-CR9, CR9-CR10, DR1-DR2, DR2-DR3, DR3-DR4, DR4-DR5, DR5-DR6, DR6-DR7, DR7-DR8, DR8-DR9, DR9-DR10, DR10-DR11, DR11-DR12, ER1-ER

/tmp/ipykernel_9070/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_raw_clean_data.fif
[done]


In [7]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 160, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.


/tmp/ipykernel_9070/2669736035.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


446 events found
Event IDs: [0 1]
Using qt as 2D backend.


Channels marked as bad:
['C3', 'C4', 'CZ', 'EKG1', 'EKG2', 'EMPTY', 'EMPTY_160', 'EMPTY_161', 'EMPTY_162', 'EMPTY_163', 'EMPTY_164', 'EMPTY_165', 'EMPTY_166', 'EMPTY_167', 'EMPTY_168', 'EMPTY_169', 'EMPTY_170', 'EMPTY_171', 'EMPTY_172', 'EMPTY_173', 'EMPTY_174', 'EMPTY_175', 'EMPTY_176', 'EMPTY_177', 'EMPTY_178', 'EMPTY_179', 'EMPTY_180', 'EMPTY_181', 'EMPTY_182', 'EMPTY_183', 'EMPTY_184', 'EMPTY_185', 'EMPTY_186', 'EMPTY_187', 'EMPTY_188', 'EMPTY_189', 'EMPTY_190', 'EMPTY_191', 'EMPTY_192', 'EMPTY_193', 'EMPTY_194', 'EMPTY_195', 'EMPTY_196', 'EMPTY_197', 'EMPTY_198', 'EMPTY_199', 'EMPTY_200', 'EMPTY_201', 'EMPTY_202', 'EMPTY_203', 'EMPTY_204', 'EMPTY_205', 'EMPTY_206', 'EMPTY_207', 'EMPTY_208', 'EMPTY_209', 'EMPTY_210', 'EMPTY_211', 'EMPTY_212', 'EMPTY_213', 'EMPTY_214', 'EMPTY_215', 'EMPTY_216', 'EMPTY_217', 'EMPTY_218', 'EMPTY_219', 'EMPTY_220', 'EMPTY_221', 'EMPTY_222', 'EMPTY_223', 'EMPTY_224', 'EMPTY_225', 'EMPTY_226', 'EMPTY_227', 'EMPTY_228', 'EMPTY_229', 'EMPTY_230', 'EMPTY_23

In [8]:
# # From interactive sessions
# interactive_annot = bp_filtered_fif.annotations
# for x in range(0, len(bp_filtered_fif.annotations)):
#     print(interactive_annot[x])
    
    
# bp_filtered_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True) 

OrderedDict([('onset', 10.557), ('duration', 3.0889997482299805), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 16.75), ('duration', 7.395999908447266), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 30.469246), ('duration', 0.7965545654296875), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 86.897003), ('duration', 2.3969955444335938), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 110.702003), ('duration', 113.8919906616211), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 136.672897), ('duration', 0.3615570068359375), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 148.435486), ('duration', 1.1920166015625), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 151.757294), ('duration', 0.4123992919921875), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 161.135864), ('duration', 1.1298675537109375), ('description', 'B

/tmp/ipykernel_9070/1805030964.py:7: RuntimeWarning: This filename (/home/brooke/pacman/raw_data/BJH039/ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  bp_filtered_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True)


## Epoching the data 

### Onset

In [9]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.


/tmp/ipykernel_9070/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


446 events found
Event IDs: [0 1]
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 32001 original time points ...
Splitting into 2 parts
Loading data for 112 events and 32001 original time points ...


/tmp/ipykernel_9070/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 111 events and 32001 original time points ...


### Trial End

In [11]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.


/tmp/ipykernel_9070/875914644.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


446 events found
Event IDs: [0 1]
Not setting metadata
222 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts


/tmp/ipykernel_9070/875914644.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 111 events and 20001 original time points ...
Loading data for 111 events and 20001 original time points ...


### Last Away

In [12]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,18,66400,0,1
1,3,12,79200,0,1
2,5,1,128400,0,1
3,6,20,143200,0,1
4,7,5,165400,0,1
...,...,...,...,...,...
173,215,10,3133400,0,1
174,216,13,3144400,0,1
175,217,9,3154100,0,1
176,219,18,3222600,0,1


In [13]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [14]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.
Not setting metadata
178 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...


/tmp/ipykernel_9070/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9070/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 178 events and 20001 original time points ...


### First Dot

In [15]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,18,63700,0,1
1,3,12,77100,0,1
2,4,15,89500,0,1
3,5,1,126300,0,1
4,6,20,136100,0,1
...,...,...,...,...,...
207,216,13,3142700,0,1
208,218,6,3160400,0,1
209,219,18,3220000,0,1
210,220,15,3239400,0,1


In [16]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.
Not setting metadata
212 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Loading data for 212 events and 18001 original time points ...


/tmp/ipykernel_9070/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9070/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [18]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,14,47500,0,1
1,2,18,63100,0,1
2,3,12,76600,0,1
3,4,15,88700,0,1
4,5,1,126100,0,1
...,...,...,...,...,...
216,217,9,3154100,0,1
217,218,6,3159800,0,1
218,219,18,3219500,0,1
219,220,15,3238900,0,1


In [19]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [20]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.
Not setting metadata
221 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 111 events and 20001 original time points ...


/tmp/ipykernel_9070/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9070/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 110 events and 20001 original time points ...


## Ghost Attack

In [22]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,25200,0,1
1,7,5,165400,0,1
2,11,9,213900,0,1
3,13,4,453600,0,1
4,16,6,487400,0,1
...,...,...,...,...,...
63,206,16,3026100,0,1
64,207,14,3037700,0,1
65,209,2,3060100,0,1
66,210,5,3070600,0,1


In [23]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [24]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif...
    Range : 0 ... 2197999 =      0.000 ...  1098.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data-1.fif...
    Range : 2198000 ... 3639799 =   1099.000 ...  1819.899 secs
Ready.
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 68 events and 20001 original time points ...


/tmp/ipykernel_9070/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9070/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH039/ieeg/BJH039_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
